* This file is for cleaning and analyzing extracted data into monthly summaries 
* Then exporting to a master csv 

In [2]:
import numpy as np
import pandas as pd
import csv
import os
import datetime

In [3]:
for filename in sorted(os.listdir("./raw_data")):
    df = pd.read_csv(f"./raw_data/{filename}")
    
    #for each variable, create a temp dataset removing null values
    final = pd.DataFrame()
    #for temp
    df_temp = df[df["TEMP"] != 9999.9]
    grouped = df_temp.copy().groupby("MONTH").agg({'TEMP' : ['mean', 'median']})
    grouped.columns = [f'{col[0]}_{col[1]}' for col in grouped.columns]
    grouped = grouped.reset_index()
    final["MONTH"] = grouped["MONTH"]
    final["YEAR"] = filename[:4]
    final[["TEMP_mean", "TEMP_median"]] = grouped[["TEMP_mean", "TEMP_median"]]
    final["TEMP_var"] = df_temp.groupby("MONTH")["TEMP"].var().reset_index(drop = True)
    #min temp
    df_temp = df[df["MIN"] != 9999.9]
    grouped = df_temp.copy().groupby("MONTH").agg({'MIN' : ['min']})
    grouped.columns = [f'{col[0]}_{col[1]}' for col in grouped.columns]
    grouped = grouped.reset_index()
    final["TEMP_min"] = grouped["MIN_min"]
    #max temp
    df_temp = df[df["MAX"] != 9999.9]
    grouped = df_temp.copy().groupby("MONTH").agg({'MAX' : ['max']})
    grouped.columns = [f'{col[0]}_{col[1]}' for col in grouped.columns]
    grouped = grouped.reset_index()
    final["TEMP_max"] = grouped["MAX_max"]
    #precip
    df_temp = df[df["PRCP"] != 99.99]
    grouped = df_temp.copy().groupby("MONTH").agg({'PRCP' : ['mean', 'median', 'min', 'max']})
    grouped.columns = [f'{col[0]}_{col[1]}' for col in grouped.columns]
    grouped = grouped.reset_index()
    final[["PRCP_mean", "PRCP_median", "PRCP_min", "PRCP_max"]] = grouped[["PRCP_mean", "PRCP_median", "PRCP_min", "PRCP_max"]]
    final["PRCP_var"] = df_temp.groupby("MONTH")["PRCP"].var().reset_index(drop = True)
        
    if int(filename[:4]) == 1950:
        final.to_csv('monthly_weather_stats.csv', mode='w', index=False, header=True)     #write to CSV with header if it's the first file
    else:
        final.to_csv('monthly_weather_stats.csv', mode='a', index=False, header=False)